In [1]:
import requests
import json
import urllib.parse
import datetime
import sqlite3
import os
import re
import re 
import xml.etree.ElementTree as ET
from collections import Counter
from Bio import Entrez
import mercury as mr

In [2]:
Entrez.email = 'pethke.johanna@gmail.com'

In [3]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_jp.db")
cur = con.cursor()

In [4]:
## Tabellen Schema erstellen
#------------------- Tabelle Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Dokument(PMID INTEGER UNIQUE, Titel TEXT, Jahr_Dok YEAR)")

#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, Jahr YEAR, Anzahl INTEGER, UNIQUE(PMID,Jahr) ON CONFLICT REPLACE)")

#------------------- Tabelle MeSH --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS MeSH(UI VARCHAR, Term TEXT, Baum TEXT, Jahr_MeSH YEAR, EK INTEGER, NEK INTEGER, NEKK INTEGER, UNIQUE(UI) ON CONFLICT REPLACE)")

#------------------- Tabelle Mesh - Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS MeSH_Dok(UI VARCHAR, PMID INTEGER, EK_Dok INTEGER, NEK_Dok INTEGER, NEKK_Dok INTEGER, UNIQUE(UI,PMID) ON CONFLICT REPLACE)")
con.commit()

In [5]:
## Funktionen um Daten in DB einzupflegen

# Dokument Daten
def DB_Insert_Dokument(PMID, Titel, Jahr_Dok):
    try:
        cur.execute("INSERT INTO Dokument(PMID, Titel, Jahr_Dok) VALUES ('"+str(PMID)+"', '"+str(Titel)+"', '"+str(Jahr_Dok)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # Dokumenten-Daten schon in DB
        return
    
# Zitationsdaten für ein Dokument
def DB_Insert_Zitationen(values_str):
    try:
        cur.execute("INSERT INTO Zitationen(PMID, Jahr, Anzahl) VALUES " + str(values_str))
        con.commit()
        # Insert erfolgreich
    except:
        # Zitationsdaten schon in DB
        return
    
# MeSH Daten
def DB_Insert_MeSH_Daten(UI, Term, Baum, Jahr_MeSH):
    try:
        cur.execute("INSERT INTO MeSH(UI, Term, Baum, Jahr_MeSH) VALUES ('"+str(UI)+"', '"+str(Term)+"', '"+str(Baum)+"', '"+str(Jahr_MeSH)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH schon in DB
        return
    
# MeSH Analyse-Daten 
def DB_Insert_MeSH_Analyse(EK, NEK, NEKK, UI):
    try:
        # print("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"' WHERE UI ='" + str(UI) + "'")
        cur.execute("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"' WHERE UI ='" + str(UI) + "'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokumenten Zuordnung
def DB_Insert_MeSH_PMID(UI, PMID):
    try:
        cur.execute("INSERT INTO MeSH_Dok(UI, PMID) VALUES ('"+str(UI)+"', '"+str(PMID)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokument Analyse-Daten 
def DB_Insert_MeSH_Dokument_Analyse(EK_Dok, NEK_Dok, NEKK_Dok, UI, PMID):
    try:
        # print("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        cur.execute("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return
    

def DB_Select_Alle_MeSH():
    cur.execute("SELECT MeSH FROM MeSH")
    MeSHs = [x[0] for x in cur.fetchall()]
    return MeSHs

def DB_Select_Alle_Dokumente():
    cur.execute("SELECT PMID FROM Dokument")
    Doks = [x[0] for x in cur.fetchall()]
    return Doks

def DB_Select_Jahr_MeSH(UI):
    cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Term(UI):
    cur.execute("SELECT Term FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Jahr_Dok(PMID):
    cur.execute("SELECT Jahr_Dok FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Titel(PMID):
    cur.execute("SELECT Titel FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]


# Funktion, die prüft, ob MeSH Daten schon in DB
def pruefe_MeSH_in_DB(UI):
    cur.execute("SELECT count(1) FROM MeSH WHERE UI = '" + str(UI) + "'")
    Mesh_in_DB = cur.fetchall()[0][0]
    if Mesh_in_DB != 0:
        return 1 # Mesh ist schon in DB
    else:
        return 0 # Mesh ist noch nicht in DB

# Funktion, die prüft, ob Dokument schon in DB
def pruefe_Dok_in_DB(PMID):
    cur.execute("SELECT count(1) FROM Dokument WHERE PMID = '" + str(PMID) + "'")
    Dok_in_DB = cur.fetchall()[0][0]
    if Dok_in_DB != 0:
        return 1 # Dokument ist schon in DB
    else:
        return 0 # Dokument ist noch nicht in DB

In [6]:
## Funktionen

# Funktion ermittelt alle (UI, Term, Link), die zu Suchwort passen
def suchwort_MeSH_Auswahl(suchwort):
    url = "https://id.nlm.nih.gov/mesh/lookup/descriptor?label="+str(suchwort)+"&match=contains&year=current&limit=50"
    anfrage = requests.get(url, headers={"Accept": "application/json"})
    antwort_Daten = anfrage.json()
    Terme = [antwort['label'] for antwort in antwort_Daten] 
    Links = [antwort['resource'] for antwort in antwort_Daten]
    UIs = [re.findall("^.*mesh\/(.*)$", link)[0] for link in Links]
    if len(UIs) == 0:
        # keine Daten zu Suchwort gefunden
        return 1
    elif len(Terme) == len(Links) == len(UIs):
        MeSH_Liste_komplett = list(zip(UIs, Terme, Links))
        return MeSH_Liste_komplett
    else:
        # Fehler in Datenerhebung
        return 0
    
def getOutputForMercuryMeShListekomplett(MeSH_Liste_komplett):
    ouputForMercury = []
    for mesh in MeSH_Liste_komplett:
        item = str(mesh[0]) + ", " + str(mesh[1])
        ouputForMercury.append(item)
    return ouputForMercury

def getMeSHListekomplettNurUIs(MeSH_Liste_komplett):
    nurUIs = []
    for mesh in MeSH_Liste_komplett:
        nurUIs.append(mesh[0])
    return nurUIs

def nurNochNichtEingefuegte(Indizes, MeSH_Liste_komplett):
    MeSH_Liste_Auswahl = []
    for index in Indizes:
        if pruefe_MeSH_in_DB(MeSH_Liste_komplett[index][0]) == 0: # MeSH wird nur in DB eingepflegt, wenn es noch nicht vorhanden ist
            MeSH_Liste_Auswahl.append(MeSH_Liste_komplett[index])
        else: # MeSH ist schon in DB, muss also nicht erneut analyisiert werden
            # print("MeSH schon in DB")
            continue
    return MeSH_Liste_Auswahl

# Funktion, die zu ausgewählten MeSH weitere Daten ermittelt und diese in DB einpflegt
def MeSH_Auswahl_setzen(Indizes, MeSH_Liste_komplett):
    MeSH_Liste_Auswahl = []
    for index in Indizes:
        if pruefe_MeSH_in_DB(MeSH_Liste_komplett[index][0]) == 0: # MeSH wird nur in DB eingepflegt, wenn es noch nicht vorhanden ist
            MeSH_Liste_Auswahl.append(MeSH_Liste_komplett[index])

            url = str(MeSH_Liste_komplett[index][2])+".json"
            anfrage = requests.get(url, headers={"Accept": "application/json"})
            antwort_Daten = anfrage.json()
            TreeNumber = antwort_Daten["treeNumber"]
            if type(TreeNumber) == str:
                Baum = re.findall("^.*mesh\/(.*)$", TreeNumber)[0]
            elif type(TreeNumber) == list:
                Baum = re.findall("^.*mesh\/(.*)$", TreeNumber[0])[0]
            Jahr_MeSH = datetime.datetime.strptime(antwort_Daten["dateCreated"], "%Y-%m-%d").year
            DB_Insert_MeSH_Daten(MeSH_Liste_komplett[index][0], MeSH_Liste_komplett[index][1], Baum, Jahr_MeSH)
        else: # MeSH ist schon in DB, muss also nicht erneut analyisiert werden
            # print("MeSH schon in DB")
            continue
    return MeSH_Liste_Auswahl

# Funktion, die die Dokumente ermittelt, die mit dem MeSH verknüpft sind und vor der Aufnahme des MeSHs veröffentlicht wurden (Von 1900 an)
def Dokumente_mit_MeSH(Term, UI, count_max):
    Jahr_MeSH = DB_Select_Jahr_MeSH(UI)
    # API ist auf die ersten 10000 Dokumente limitiert; esearch ist eine Möglichkeit an alle Dokumente zu kommen (wenn es denn mehr gibt) allerdings muss man auf einer UNIX Umgebung arbeiten https://dataguide.nlm.nih.gov/edirect/edirect-vs-e-utilities.html
    # Abfrage von Dokumente, die vor der Aufnahme des MeSHs veröffentlicht wurden (Von 1900 an)
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=("+urllib.parse.quote(Term)+"[MeSH+Terms])+AND+((\"1900\"[Date+-+Publication]+%3A+\""+str(Jahr_MeSH)+"\"[Date+-+Publication]))&retmax="+str(count_max)+"&sort=relevance&retmode=json"
    # print(url)
    anfrage = requests.get(url)
    antwort_Daten = json.loads(anfrage.text)
    Dokumente_PMIDs_komplett = antwort_Daten['esearchresult']['idlist']
    # Verbindung Dokument - MeSH in DB einpflegen
    for PMID in Dokumente_PMIDs_komplett:
        DB_Insert_MeSH_PMID(UI, PMID)
    return Dokumente_PMIDs_komplett

#Funktion um XML (Roots) für Dokumente zu ermitteln 
def Roots_Dokumente(Dokumente_PMIDs_komplett): #hat für 9999 Dokumente ca 1,5 Minuten gedauert
    roots = []
    begin = 0
    end = 300
    if len(Dokumente_PMIDs_komplett) > 300:
        while end < len(Dokumente_PMIDs_komplett):
            Dokumente_PMID_str = ','.join(Dokumente_PMIDs_komplett[begin:end])
            url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+urllib.parse.quote(str(Dokumente_PMID_str))
            anfrage = requests.get(url)
            root = ET.fromstring(anfrage.text)
            roots.append(root)
            end = end + 300
            begin = begin + 300
    Dokumente_PMID_str = ','.join(Dokumente_PMIDs_komplett[begin:])
    url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+urllib.parse.quote(str(Dokumente_PMID_str))
    anfrage = requests.get(url)
    root = ET.fromstring(anfrage.text)
    roots.append(root)
    return roots

# Funktion, die ermittelt, welche Dokumente noch analyisiert werden müssen (noch nicht in DB)
def Dokumente_Zu_Analyisieren(Dokumente_PMIDs_komplett):
    Dokumente_Zu_Analysieren = []
    for PMID in Dokumente_PMIDs_komplett:
        if pruefe_Dok_in_DB(PMID) == 0:
            Dokumente_Zu_Analysieren.append(PMID)
    return Dokumente_Zu_Analysieren

# Funktion zur Ermittlung der Daten für Dokumente (Titel, Veröffentlichungsjahr) und das Einfügen in die DB 
def Daten_Dokument(Dokumente_zu_Analysieren):
    roots = Roots_Dokumente(Dokumente_zu_Analysieren)
    for root in roots:
        root_Dokument_Daten = root.findall('./PubmedArticle')
        for Dok_XML in root_Dokument_Daten:
            PMID = Dok_XML.find('.MedlineCitation/PMID')
            Jahr_Dok = Dok_XML.find('./PubmedData/History/PubMedPubDate[@PubStatus="pubmed"]/Year')
            Titel = Dok_XML.find('./MedlineCitation/Article/ArticleTitle')
            DB_Insert_Dokument(PMID.text, Titel.text, Jahr_Dok.text)

# Funktion um alle Dokumente zu ermitteln, die ein bestimmtes Dokument zitieren (PMID)
def zitierende_Dokumente(PMID):
    zitierende_Dokumente = []
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&retmode=json&id="+urllib.parse.quote(str(PMID))
    # print(url)
    anfrage = requests.get(url)
    antwort = json.loads(anfrage.text)
    # print(json_data)
    if "linksets" in antwort:
        if "linksetdbs" in antwort["linksets"][0]:
            if "links" in antwort["linksets"][0]["linksetdbs"][0]:
                zitierende_Dokumente = antwort["linksets"][0]["linksetdbs"][0]["links"]
    return zitierende_Dokumente

# Funktion zur Ermittlung des Zitations-Histogramms (Verteilung der Zitationen über die Jahre) 
def ZitationsHistogramm(roots):
    Jahre = []
    histogram = Counter()
    for root in roots:
        Jahr_Dok = root.findall('./PubmedArticle/PubmedData/History/PubMedPubDate[@PubStatus="pubmed"]/Year')
        Jahre = [int(pub.text) for pub in Jahr_Dok]
        histogram = histogram + Counter(Jahre)
    return histogram

# Funktion zur Ermittlung der Zitationsdaten für Dokumente und das Einfügen in die DB
def Zitationsdaten(Dokumente_zu_Analysieren):
    for PMID in Dokumente_zu_Analysieren:
        zitierende_Dokumente_Liste = zitierende_Dokumente(PMID)
        if len(zitierende_Dokumente_Liste) == 0:
            # Es liegen keine Zitationsdaten vor
            continue
        else:
            roots_zitierende_Dokumente = Roots_Dokumente(zitierende_Dokumente_Liste)
            Histogramm = ZitationsHistogramm(roots_zitierende_Dokumente)
            values_str = ""
            for Jahr in Histogramm:
                values_str = values_str + ", ('" +str(PMID) + "', '" + str(Jahr) +"', '" + str(Histogramm[Jahr]) + "')"
            values_str = values_str[1:]
            DB_Insert_Zitationen(values_str)

# Funktion zur Ermittlung der Anzahl der Zitationen in einem bestimmten Zeitraum
def Zitationen_Von_Bis(PMID, von, bis=2023):
    url = "SELECT sum(Anzahl) FROM Zitationen WHERE PMID = "+str(PMID)+" and Jahr >= '"+str(von)+ "' and Jahr <= '"+str(bis)+ "'"
    cur.execute(url)
    antwort = cur.fetchall()[0][0] 
    Anzahl = antwort if antwort != None else 0 
    return Anzahl

# Funktion zur Analyse der Daten (Emergenz) für ein bestimmtes MeSH
def Analyse(UI):
    EK = 0
    NEK = 0
    NEKK = 0
    # Ermittlung des Einführungsjahres des MeSH
    Jahr_MeSH = DB_Select_Jahr_MeSH(UI)
    # Ermittlung zugehöriger Dokumentemesh
    cur.execute("SELECT PMID FROM MeSH_Dok WHERE UI = '" + str(UI)+ "'")
    PMIDs = [x[0] for x in cur.fetchall()]
    for PMID in PMIDs:
        # Ermittlung Veröffentlichungsdaten des Dokuments
        try:
            Jahr_Dok = DB_Select_Jahr_Dok(PMID)
        except:
            continue
        Zeitraum = Jahr_MeSH - Jahr_Dok + 1
        EK_Dok = Zitationen_Von_Bis(PMID, Jahr_Dok, Jahr_MeSH) # Emergent: Veröffentlichung Dokument bis einschließlich Einführungsjahr MeSH
        EK = EK + EK_Dok
        NEK_Dok = Zitationen_Von_Bis(PMID, Jahr_MeSH+1) # Nicht Emergent: Einführungsjahr MeSH + 1 bis heute
        NEK = NEK + NEK_Dok
        NEKK_Dok = Zitationen_Von_Bis(PMID, Jahr_MeSH+1, Jahr_MeSH+Zeitraum) # Nicht Emergent Korrigiert: Zeitraum nach Einführung MeSH angepasst
        NEKK = NEKK + NEKK_Dok
        DB_Insert_MeSH_Dokument_Analyse(EK_Dok, NEK_Dok, NEKK_Dok, UI, PMID)
    DB_Insert_MeSH_Analyse(EK, NEK, NEKK, UI)
    return 1


In [ ]:
show_code = mr.Checkbox(label="Quelltext ein-/ausblenden", value=True)
app = mr.App(title="Datenerhebung", 
             description="Erheben Sie entweder Daten zu Themengebieten, oder lassen Sie sich schon analysierte Ergebnisse anzeigen:", show_code=show_code.value)

In [ ]:
anzahlDok = mr.Slider(value=20, min=10, max=1000, label="Anzahl Dokumente pro MeSH", step=10)
suchwort = mr.Text(label="Suchen Sie anhand eines Suchwortes nach MeSHs", value="Suchbegriff")

mr.Markdown(f"## Geben Sie in nebenstehendes Textfeld einen Begriff ein, zu dem Sie zusammenstehende MeSH finden möchten.")
mr.Markdown(f"### Suchbegriff: **{suchwort.value}**")
mr.Markdown(f"## Wählen Sie nun aus nebenstehender Liste die MeSH aus, für die Sie eine Analyse durchführen möchten.")

alleMeSH = mr.Checkbox(value=False, label="Alle MeSH auswählen")

MeSH_Liste_komplett = suchwort_MeSH_Auswahl({suchwort.value})
# Fehlerbehandlung -> Abbruch
if MeSH_Liste_komplett == 0:
    mr.Markdown(f"Fehler in Datenerhebung")
    mr.Stop()
elif MeSH_Liste_komplett == 1:      
    mr.Markdown(f"Kein passendes MeSH zum Suchwort gefunden")
    mr.Stop()
MeSH_Liste_komplett_nurUIs = getMeSHListekomplettNurUIs(MeSH_Liste_komplett)
MeSH_Liste = getOutputForMercuryMeShListekomplett(MeSH_Liste_komplett)

if alleMeSH.value == False:
    selected = mr.MultiSelect(label="Wählen Sie die MeSH aus, für die eine Analyse durchgeführt werden soll.", value=[], choices=MeSH_Liste) 
    button_pruefeAuswahl = mr.Button(label="MeSH Auswahl prüfen", style="primary")
    button_MeSH_Auswahl = mr.Button(label="Analyse starten", style="primary")
    if selected.value != []:
        mr.Markdown(f"### **Folgende MeSH sind ausgewählt:** ")
        for mesh in selected.value:
            mr.Markdown(f"#### {mesh}")  
else:
    mr.Markdown(f"### **Folgende MeSH stehen mit dem Suchwort in Verbindung:** ")
    selected = MeSH_Liste
    for mesh in MeSH_Liste:
        mr.Markdown(f"#### {mesh}")    
    button_pruefeAuswahl = mr.Button(label="MeSH Auswahl prüfen", style="primary")
    button_MeSH_Auswahl = mr.Button(label="Analyse starten", style="primary")


if button_pruefeAuswahl.clicked:
    # Liste zur übersichtlichen Ausgabe
    mesh_List = []
    if type(selected) != list:
        for mesh in selected.value:
            mesh_List.append(mesh.split(", "))
            
    else:
        for i in range(len(selected)):
            mesh_List.append(selected[i].split(", "))
    # mr.Markdown(f"### **Ausgewählte MeSH:** ")
    # Identifikation der Indizes, Ausgabe der ausgewähltens MeSH
    indizes = []
    for MeSH in mesh_List:
        indizes.append(MeSH_Liste_komplett_nurUIs.index(MeSH[0]))
        # mr.Markdown(f"### - {MeSH}")
    MeSH_Liste_Auswahl = nurNochNichtEingefuegte(indizes, MeSH_Liste_komplett)
    # Identifikation der MeSH, die noch nicht analysiert wurden
    mr.Markdown(f"### **MeSH, die noch nicht analysiert wurden:** ")
    if MeSH_Liste_Auswahl == []:
        mr.Markdown(f"### Die ausgewählten MeSHs wurden bereits alle analysiert.")
    else:
        for MeSH in MeSH_Liste_Auswahl:
            mr.Markdown(f"#### - {MeSH[0], MeSH[1]}")
    
if button_MeSH_Auswahl.clicked:
    MeSH_Liste_Auswahl = MeSH_Auswahl_setzen(indizes, MeSH_Liste_komplett)
    mr.Markdown(f"### Es werden nun zu den noch nicht analysierten MeSH {anzahlDok.value} Dokumente ermittelt und in die Datenbank eingepflegt")
    for MeSH in MeSH_Liste_Auswahl:
        try:
            mr.Markdown(f"#### Datenerhebung für MeSH: {MeSH[0], MeSH[1]}")
            # In Zusammenhang stehende Dokumente ermitteln und in MeSH_Dok einpflegen
            Dokumente_PMIDs_komplett = Dokumente_mit_MeSH(MeSH[1], MeSH[0], anzahlDok.value)
            # Auswahl der Dokumente, zu denen noch keine Daten erhoben wurden
            Dokumente_Auswahl_Analyse = Dokumente_Zu_Analyisieren(Dokumente_PMIDs_komplett)
            # Daten der Dokumente ermitteln und einpflegen
            Daten_Dokument(Dokumente_Auswahl_Analyse)
            # Zitationsdaten erheben und einpflegen
            Zitationsdaten(Dokumente_Auswahl_Analyse)
            mr.Markdown(f"#### Kennzahlanalyse wird durchgeführt.")
            # Durchführung der Analyse Kennzahlen
            done = Analyse(MeSH[0])
            if done == 1:
                mr.Markdown(f"## Die Analyse wurde erfolgreich durchgeführt.")
        except:
            # Datenerhebung oder Analyse fehlgschlagen, alle zu dem MeSH bisher gespeicherten Daten werden aus der Datenbank entfernt
            cur.execute("DELETE FROM MeSH WHERE UI = '" + str(MeSH[0]) + "'") 
            cur.execute("DELETE FROM MeSH_Dok WHERE UI = '" + str(MeSH[0]) + "'") 
            cur.execute("DELETE FROM MeSH WHERE COALESCE (UI, Term, Baum, Jahr_MeSH, EK, NEK, NEKK) IS NULL;")
            cur.execute("DELETE FROM Dokument WHERE COALESCE (PMID, Ttitle, Jahr_Dok) IS NULL;")
            cur.execute("DELETE FROM MeSH_Dok WHERE COALESCE (UI, PMID, EK_Dok, NEK_Dok, NEKK_Dok) IS NULL;")
            cur.execute("DELETE FROM Zitationen WHERE COALESCE (PMID, Jahr, Anzahl) IS NULL;")
            cur.commit()